In [1]:
import pandas as pd
import numpy as np
from prophet import Prophet
from prophet.diagnostics import cross_validation, performance_metrics
from prophet.plot import plot_plotly, plot_components_plotly
from pathlib import Path
import joblib
import json
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import warnings
import plotly.offline as py
py.init_notebook_mode(connected=True) # Enables interactive plots in the notebook

warnings.filterwarnings('ignore')
 
# ---------------------------------------------
# Paths
# ---------------------------------------------
FEATURE_PATH = Path("D:\\sales_analytics_project\\data\\processed\\featurized_sales_data.csv")
MODEL_PATH = Path("D:\\sales_analytics_project\\models\\prophet_model.pkl")
FORECAST_CSV = Path("D:\\sales_analytics_project\\data\\processed\\prophet_forecast.csv")
METADATA_PATH = Path("D:\\sales_analytics_project\\models\\prophet_metadata.json")
TARGET = "revenue_npr"
FORECAST_DAYS = 90

# ---------------------------------------------
# Model Evaluation
# ---------------------------------------------
def evaluate_model(y_true, y_pred, model_name="Prophet"):
    """Calculate comprehensive evaluation metrics"""
    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    r2 = r2_score(y_true, y_pred)
    
    mask = y_true != 0
    mape = np.mean(np.abs((y_true[mask] - y_pred[mask]) / y_true[mask])) * 100 if mask.sum() > 0 else 0
    
    metrics = {
        "model": model_name,
        "MAE": float(mae),
        "RMSE": float(rmse),
        "R2": float(r2),
        "MAPE": float(mape)
    }
    
    print(f"\n{'='*50}")
    print(f"{model_name} Model Evaluation Metrics")
    print(f"{'='*50}")
    print(f"MAE:  {mae:,.2f} NPR")
    print(f"RMSE: {rmse:,.2f} NPR")
    print(f"R²:   {r2:.4f}")
    print(f"MAPE: {mape:.2f}%")
    print(f"{'='*50}\n")
    
    return metrics

def remove_outliers(df, column, n_std=3):
    """Remove outliers using z-score method"""
    mean = df[column].mean()
    std = df[column].std()
    df_clean = df[np.abs(df[column] - mean) <= n_std * std].copy()
    removed = len(df) - len(df_clean)
    if removed > 0:
        print(f"Removed {removed} outliers from {column}")
    return df_clean

# ---------------------------------------------
# Prepare Data for Prophet
# ---------------------------------------------
def prepare_prophet_data(df, test_size=0.2):
    """Prepare data with train/test split"""
    # Aggregate to daily
    df_daily = df.groupby("date").agg({
        TARGET: "sum",
        "rainfall_mm": "mean",
        "temperature": "mean",
        "discount_percent": "mean",
        "customer_traffic": "sum",
        "festival_season": "max"
    }).reset_index()
    
    # Remove outliers
    df_daily = remove_outliers(df_daily, TARGET, n_std=3)
    
    # Rename for Prophet
    df_prophet = df_daily.rename(columns={"date": "ds", TARGET: "y"})
    
    # Prepare regressors
    regressor_cols = ["rainfall_mm", "temperature", "discount_percent", "customer_traffic", "festival_season"]
    # Ensure all regressor columns exist, fill with 0 if not
    for col in regressor_cols:
        if col not in df_prophet.columns:
            df_prophet[col] = 0
            
    df_regressors = df_prophet[["ds"] + regressor_cols].copy()
    
    # Train/test split (time-based)
    split_index = int(len(df_prophet) * (1 - test_size))
    train_df = df_prophet.iloc[:split_index][["ds", "y"]].copy()
    test_df = df_prophet.iloc[split_index:][["ds", "y"]].copy()
    
    return df_prophet, df_regressors, train_df, test_df

# ---------------------------------------------
# Train Prophet Model
# ---------------------------------------------
def train_prophet_model(train_df, df_regressors, test_df=None):
    """Train Prophet with optimized settings"""
    print("\nInitializing Prophet model with enhanced settings...")
    
    m = Prophet(
        yearly_seasonality=True,
        weekly_seasonality=True,
        daily_seasonality=False,
        seasonality_mode='multiplicative',
        changepoint_prior_scale=0.3,
        seasonality_prior_scale=10.0,
        interval_width=0.95
    )
    
    m.add_seasonality(name='monthly', period=30.5, fourier_order=5)
    
    regressor_cols = [col for col in df_regressors.columns if col != "ds"]
    for col in regressor_cols:
        m.add_regressor(col)
        print(f"  Added regressor: {col}")
    
    train_with_regressors = train_df.merge(df_regressors, on="ds", how="left")
    
    print(f"\nTraining Prophet on {len(train_with_regressors)} days...")
    m.fit(train_with_regressors)
    print("✓ Training complete")
    
    test_metrics = None
    if test_df is not None and len(test_df) > 0:
        print("\nEvaluating on test set...")
        test_with_regressors = test_df.merge(df_regressors, on="ds", how="left")
        forecast_test = m.predict(test_with_regressors)
        
        y_true = test_df['y'].values
        y_pred = forecast_test['yhat'].values
        test_metrics = evaluate_model(y_true, y_pred, "Prophet")
    
    return m, test_metrics

# ---------------------------------------------
# Generate Future Forecast
# ---------------------------------------------
def generate_forecast(model, df_regressors, periods=90):
    """Generate future forecast with regressors"""
    print(f"\nGenerating {periods}-day forecast...")
    
    future = model.make_future_dataframe(periods=periods)
    
    last_date = df_regressors['ds'].max()
    future_dates = pd.date_range(start=last_date + pd.DateOffset(days=1), periods=periods)
    
    future_regressors = []
    for future_date in future_dates:
        same_month = df_regressors[df_regressors['ds'].dt.month == future_date.month]
        
        if len(same_month) > 0:
            future_regressors.append({
                'ds': future_date,
                'rainfall_mm': same_month['rainfall_mm'].mean(),
                'temperature': same_month['temperature'].mean(),
                'customer_traffic': same_month['customer_traffic'].mean(),
                'discount_percent': 0,
                'festival_season': 0
            })
        else:
            future_regressors.append({
                'ds': future_date,
                'rainfall_mm': df_regressors['rainfall_mm'].mean(),
                'temperature': df_regressors['temperature'].mean(),
                'customer_traffic': df_regressors['customer_traffic'].mean(),
                'discount_percent': 0,
                'festival_season': 0
            })
    
    df_future_regressors = pd.DataFrame(future_regressors)
    all_regressors = pd.concat([df_regressors, df_future_regressors], ignore_index=True)
    
    future_with_regressors = future.merge(all_regressors, on="ds", how="left")
    future_with_regressors = future_with_regressors.ffill().bfill()
    
    forecast = model.predict(future_with_regressors)
    
    print("✓ Forecast generated")
    
    return forecast

# ---------------------------------------------
# Main Execution Function
# ---------------------------------------------
def run_prophet_pipeline(test_size=0.2):
    print("="*60)
    print("IMPROVED PROPHET FORECASTING MODEL")
    print("="*60)
    
    print("\n[1/5] Loading featurized data...")
    df = pd.read_csv(FEATURE_PATH, parse_dates=["date"])
    
    print("\n[2/5] Preparing Prophet data...")
    df_prophet, df_regressors, train_df, test_df = prepare_prophet_data(df, test_size)
    print(f"Training set: {len(train_df)} days, Test set: {len(test_df)} days")
    
    print("\n[3/5] Training Prophet model and evaluating on test set...")
    model, test_metrics = train_prophet_model(train_df, df_regressors, test_df)
    
    print("\n[4.1/5] Retraining on full dataset for production...")
    full_data = df_prophet[["ds", "y"]].merge(df_regressors, on="ds", how="left")
    
    regressor_cols = [col for col in df_regressors.columns if col != "ds"]
    m_prod = Prophet(
        yearly_seasonality=True,
        weekly_seasonality=True,
        daily_seasonality=False,
        seasonality_mode='multiplicative',
        changepoint_prior_scale=0.5
    )
    m_prod.add_seasonality(name='monthly', period=30.5, fourier_order=5)
    for col in regressor_cols:
        m_prod.add_regressor(col)
    
    m_prod.fit(full_data)
    print("✓ Production model trained.")
    
    print(f"\n[4.2/5] Saving production model...")
    MODEL_PATH.parent.mkdir(parents=True, exist_ok=True)
    joblib.dump(m_prod, MODEL_PATH)
    print(f"✓ Saved model to {MODEL_PATH}")
    
    print(f"\n[5/5] Generating {FORECAST_DAYS}-day future forecast...")
    forecast = generate_forecast(m_prod, df_regressors, FORECAST_DAYS)
    
    forecast_output = forecast[["ds", "yhat", "yhat_lower", "yhat_upper"]].copy()
    forecast_output.to_csv(FORECAST_CSV, index=False)
    print(f"✓ Saved forecast to {FORECAST_CSV}")
    
    print("\n--- Prophet Pipeline Complete! ---")
    
    # Return model and forecast for plotting
    return m_prod, forecast

# ---------------------------------------------
# RUN THE PIPELINE
# ---------------------------------------------
model, forecast_df = run_prophet_pipeline(test_size=0.2)

# ---------------------------------------------
# ANALYSIS: PLOT THE DIAGRAMS
# ---------------------------------------------
print("\n\n" + "="*70)
print("ANALYSIS: PLOTTING FORECAST DIAGRAMS")
print("="*70)

# --- Diagram 1: Interactive Forecast Plot ---
print("\n--- Diagram 1: Interactive Forecast Plot ---")
print("This plot shows the historical data (black dots), the model's prediction (blue line), and the uncertainty interval (light blue area).")
fig_forecast = plot_plotly(model, forecast_df)
fig_forecast.update_layout(title="Prophet Forecast with Uncertainty")
py.iplot(fig_forecast)


# --- Diagram 2: Model Components Plot ---
print("\n--- Diagram 2: Model Components Plot ---")
print("These plots show the different parts of the model:")
print(" - 'trend': The overall long-term direction of the data.")
print(" - 'yearly': The seasonal pattern over a year.")
print(" - 'weekly': The seasonal pattern over a week.")
print(" - 'monthly': The custom monthly pattern we added.")
fig_components = plot_components_plotly(model, forecast_df)
py.iplot(fig_components)

IMPROVED PROPHET FORECASTING MODEL

[1/5] Loading featurized data...

[2/5] Preparing Prophet data...
Removed 1 outliers from revenue_npr
Training set: 292 days, Test set: 73 days

[3/5] Training Prophet model and evaluating on test set...

Initializing Prophet model with enhanced settings...


15:25:58 - cmdstanpy - INFO - Chain [1] start processing


  Added regressor: rainfall_mm
  Added regressor: temperature
  Added regressor: discount_percent
  Added regressor: customer_traffic
  Added regressor: festival_season

Training Prophet on 292 days...


15:25:59 - cmdstanpy - INFO - Chain [1] done processing
15:25:59 - cmdstanpy - INFO - Chain [1] start processing


✓ Training complete

Evaluating on test set...

Prophet Model Evaluation Metrics
MAE:  1,555,390.12 NPR
RMSE: 2,003,333.94 NPR
R²:   0.0551
MAPE: 29.28%


[4.1/5] Retraining on full dataset for production...


15:25:59 - cmdstanpy - INFO - Chain [1] done processing


✓ Production model trained.

[4.2/5] Saving production model...
✓ Saved model to D:\sales_analytics_project\models\prophet_model.pkl

[5/5] Generating 90-day future forecast...

Generating 90-day forecast...
✓ Forecast generated
✓ Saved forecast to D:\sales_analytics_project\data\processed\prophet_forecast.csv

--- Prophet Pipeline Complete! ---


ANALYSIS: PLOTTING FORECAST DIAGRAMS

--- Diagram 1: Interactive Forecast Plot ---
This plot shows the historical data (black dots), the model's prediction (blue line), and the uncertainty interval (light blue area).



--- Diagram 2: Model Components Plot ---
These plots show the different parts of the model:
 - 'trend': The overall long-term direction of the data.
 - 'yearly': The seasonal pattern over a year.
 - 'weekly': The seasonal pattern over a week.
 - 'monthly': The custom monthly pattern we added.
